In [12]:
import os
import re
import pandas as pd
import numpy as np
import pickle as pickle

#### 사용법
1. prediction 폴더에 submission 파일들을 넣는다
2. submission 파일명 규칙은 모델명_submission.csv로 한다.

In [13]:
prediction_dir_file_list = os.listdir("./prediction")
prediction_dir_file_list

['박submission.csv',
 '3submission.csv',
 '.ipynb_checkpoints',
 '김submission.csv',
 '1submission.csv',
 '2submission.csv']

In [14]:
submission_list = []
for file in prediction_dir_file_list:
    if bool(re.findall("submission.csv",file)):
        submission_list.append(file)
submission_list

['박submission.csv',
 '3submission.csv',
 '김submission.csv',
 '1submission.csv',
 '2submission.csv']

In [15]:
n = len(submission_list)

probs = np.array([eval(x) for x in pd.read_csv(os.path.join('./prediction',submission_list[0]))['probs']]) * (1/n)

for file in submission_list[1:]:
    submission = pd.read_csv(os.path.join('./prediction',file))
    probs += np.array([eval(x) for x in submission['probs']]) * (1/n)

probs

array([[1.72375157e-02, 9.72540653e-04, 5.24512571e-02, ...,
        6.45458051e-05, 1.43813163e-03, 8.62423163e-05],
       [2.15909148e-02, 2.74811922e-04, 1.38835244e-04, ...,
        1.30889498e-04, 2.77415216e-04, 1.19700153e-04],
       [9.99119520e-01, 2.20338361e-05, 2.52962509e-05, ...,
        2.57184226e-06, 6.16178560e-06, 6.09484941e-06],
       ...,
       [1.87324108e-04, 9.99373281e-01, 2.34945523e-05, ...,
        7.06621609e-06, 1.02438861e-04, 3.62739959e-06],
       [8.73204547e-01, 2.63710203e-04, 9.15341770e-05, ...,
        8.29995162e-05, 1.04624101e-04, 8.73736528e-05],
       [9.98586452e-01, 4.91811385e-05, 2.00932931e-05, ...,
        5.31836794e-06, 7.92130395e-06, 1.21606668e-05]])

In [16]:
result = np.argmax(probs, axis=-1)
result

array([ 3, 12,  0, ...,  1,  0,  0])

In [17]:
def num_to_label(label):
  """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
  """
  origin_label = []
  with open('dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)
  for v in label:
    origin_label.append(dict_num_to_label[v])
  
  return origin_label

In [18]:
labels = num_to_label(result)

In [19]:
probs = [  '['+','.join([format(x,'.2f') for x in softmax])+']' for softmax in probs]
probs[0]

'[0.02,0.00,0.05,0.91,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]'

In [20]:
output = pd.DataFrame({'id':submission['id'],'pred_label':labels,'probs':probs,})

In [21]:
output

,id,pred_label,probs
0,0,org:product,"[0.02,0.00,0.05,0.91,0.00,0.01,0.00,0.00,0.00,..."
1,1,per:alternate_names,"[0.02,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.00,..."
2,2,no_relation,"[1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."
3,3,no_relation,"[0.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."
4,4,no_relation,"[0.73,0.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."
...,...,...,...
7760,7760,org:place_of_headquarters,"[0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.83,0.00,..."
7761,7761,no_relation,"[1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."
7762,7762,org:top_members/employees,"[0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."
7763,7763,no_relation,"[0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,..."


In [22]:
output.to_csv('./prediction/final_submission.csv', index=False)